## Actual Working Script

In [1]:
!pip install gspread
!pip install oauth2client
!pip install -U snowflake-connector-python
!pip install -U snowflake-snowpark-python
!pip install --upgrade psycopg2
!pip install -U sqlalchemy
!pip install df2gspread
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [snowflake-snowpark-python]e-snowpark-python]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 50.6 MB/s  0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.42
    Uninstalling SQLAlchemy-2.0.42:
      Successfully uninstalled SQLAlchemy-2.0.42
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'df2gspread' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'df2gspread'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=12

In [4]:
from common_functions import ret_metabase, upload_dataframe_to_snowflake, snowflake_query
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
import sqlalchemy
import psycopg2
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import os
import requests
import json
import pandas as pd
import numpy as np
import re
import datetime
import seaborn as sns
from io import StringIO, BytesIO
import sys
from datetime import datetime



In [8]:
import requests

def address_details(lat, lon):
    # Construct the API URL
    url = f"https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}&lon={lon}"
    
    # Headers to avoid being blocked and to force English response
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; MyGeocodingApp/1.0; +http://example.com/my-geocoding-app)',
        'Accept-Language': 'en'  # Force response in English
    }
    
    # Make the API request
    # Check if the request was successful
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            display_name = data.get("display_name")
            name = data.get("name")
            road = data.get("address", {}).get("road")
            suburb = data.get("address", {}).get("suburb")
            city = data.get("address", {}).get("city") or data.get("address", {}).get("town") or data.get("address", {}).get("village")
            state = data.get("address", {}).get("state")
            country = data.get("address", {}).get("country")
            return display_name, name, road, suburb, city, state, country
        else:
            print(f"API error: status code {response.status_code}")
            return None, None, None, None, None, None, None
    except Exception as e:
        print(f"Exception during API call: {e}")
        return None, None, None, None, None, None, None

# Morocco

In [9]:
df = ret_metabase("Morocco" , 18371)

/home/ec2-user/service_account_key.json


In [10]:
df 

MAIN_SYSTEM_ID     ID   LATITUDE  LONGITUDE
0              63798  63131  30.414714  -9.496111
1              70300  69576  32.350981  -7.946858
2              70917  70192  31.526099  -9.101299
3              66030  65341  28.614859  -9.442440
4              65437  64757  33.002207  -7.624329
...              ...    ...        ...        ...
4355           74633  73898  31.754831  -8.027131
4356           76566  75824  35.768818  -5.817978
4357           81275  80517  34.228435  -3.347852
4358           76046  75304  35.616933  -5.273915
4359           76103  75361  34.007489  -5.006768

[4360 rows x 4 columns]

In [ ]:
df_empty = pd.DataFrame({
    "MAIN_SYSTEM_ID": pd.Series(dtype="Int64"),   # nullable int
    "RETAILER_ID": pd.Series(dtype="Int64"),      # nullable int
    "ADDRESS": pd.Series(dtype="string"),
    "DISTRICT": pd.Series(dtype="string"),
    "STATE": pd.Series(dtype="string"),
    "COUNTRY": pd.Series(dtype="string")
})


In [ ]:
import pandas as pd
import requests
from time import sleep
from tqdm import tqdm
import time
from datetime import datetime, timedelta

# Initialize an empty list to store the updated data
updated_data = []
print(f"Total retailers to process: {len(df)}")

# Process retailers in batches of 50
batch_size = 50
total_batches = (len(df) + batch_size - 1) // batch_size

# Start timing
start_time = time.time()

# Main progress bar for batches
with tqdm(total=len(df), desc="Processing retailers", unit="retailer") as pbar:
    for batch_num, start in enumerate(range(0, len(df), batch_size), 1):
        batch = df.iloc[start:start + batch_size]
        
        # Progress message for current batch
        print(f"\n📦 Processing batch {batch_num}/{total_batches} (retailers {start+1}-{min(start+batch_size, len(df))})")
        
        for index, row in batch.iterrows():
            main_system_id = row['MAIN_SYSTEM_ID']
            retailer_id = row['ID']
            lat = row['LATITUDE']
            lon = row['LONGITUDE']
            
            try:
                display_name, name, road, suburb, city, state, country = address_details(lat, lon)
            except Exception as e:
                print(f"❌ Error processing retailer_id {retailer_id}: {e}")
                display_name, name, road, suburb, city, state, country = [None] * 7
            
            updated_data.append([
                 main_system_id, retailer_id, display_name, city, state, country
            ])
            
            # Update progress bar
            pbar.update(1)
            pbar.set_postfix({
                'Current': f"ID:{retailer_id}",
                'Processed': len(updated_data),
                'Remaining': len(df) - len(updated_data)
            })

        # Create DataFrame for current batch only (not accumulated data)
        current_batch_data = []
        for index, row in batch.iterrows():
            main_system_id = row['MAIN_SYSTEM_ID']
            retailer_id = row['ID']
            lat = row['LATITUDE']
            lon = row['LONGITUDE']
            
            try:
                display_name, name, road, suburb, city, state, country = address_details(lat, lon)
            except Exception as e:
                print(f"❌ Error processing retailer_id {retailer_id}: {e}")
                display_name, name, road, suburb, city, state, country = [None] * 7
            
            current_batch_data.append([
                 main_system_id, retailer_id, display_name, city, state, country
            ])
        
        # Upload current batch to Snowflake immediately
        print(f"💾 Uploading batch {batch_num} to Snowflake...")
        try:
            df_current_batch = pd.DataFrame(current_batch_data, columns=[
               'main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
            now = datetime.now() + timedelta(hours=3)
            df_current_batch['dispatched_at'] = now
            upload_dataframe_to_snowflake("Morocco", df_current_batch, "fintech", "retailer_addresses_v2", "append", auto_create_table=True, conn=None)
            print(f"✅ Batch {batch_num} successfully uploaded to Snowflake!")
        except Exception as e:
            print(f"❌ Error uploading batch {batch_num} to Snowflake: {e}")
            # Continue processing but log the error
        
        # Show batch completion status
        elapsed_time = time.time() - start_time
        avg_time_per_retailer = elapsed_time / len(updated_data)
        estimated_remaining = avg_time_per_retailer * (len(df) - len(updated_data))
        
        print(f"✅ Batch {batch_num} completed. Progress: {len(updated_data)}/{len(df)} retailers ({len(updated_data)/len(df)*100:.1f}%)")
        print(f"⏱️  Elapsed: {elapsed_time:.1f}s | Est. remaining: {estimated_remaining:.1f}s")

# Calculate final timing
total_time = time.time() - start_time
print(f"\n🎉 Processing completed!")
print(f"📊 Final stats:")
print(f"   • Total retailers processed: {len(updated_data)}")
print(f"   • Total time: {total_time:.2f} seconds")
print(f"   • Average time per retailer: {total_time/len(updated_data):.3f} seconds")

# --- Final completion message ---
print(f"\n🎉 All batches processed and uploaded to Snowflake!")
print(f"📊 Final stats:")
print(f"   • Total retailers processed: {len(updated_data)}")
print(f"   • Total time: {total_time:.2f} seconds")
print(f"   • Average time per retailer: {total_time/len(updated_data):.3f} seconds")
print(f"   • All data successfully uploaded to Snowflake table: reatiler_addresses_v2")

# Optionally save to Excel as backup
# df_updated = pd.DataFrame(updated_data, columns=['main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
# df_updated.to_excel('retailers_with_regions.xlsx', index=False)
# print("✅ Exported final file: retailers_with_regions.xlsx")

Total retailers to process: 4360


Processing retailers:   0%|          | 0/4360 [00:00<?, ?retailer/s]


📦 Processing batch 1/88 (retailers 1-50)


Processing retailers:   1%|          | 50/4360 [04:55<7:13:20,  6.03s/retailer, Current=ID:66537.0, Processed=50, Remaining=4310]

💾 Uploading batch 1 to Snowflake...
/home/ec2-user/service_account_key.json
✅ Batch 1 successfully uploaded to Snowflake!
✅ Batch 1 completed. Progress: 50/4360 retailers (1.1%)
⏱️  Elapsed: 303.8s | Est. remaining: 26184.0s

📦 Processing batch 2/88 (retailers 51-100)


...

Processing retailers:  30%|██▉       | 1300/4360 [2:12:33<5:07:56,  6.04s/retailer, Current=ID:80250.0, Processed=1300, Remaining=3060]

💾 Uploading batch 26 to Snowflake...
/home/ec2-user/service_account_key.json
✅ Batch 26 successfully uploaded to Snowflake!
✅ Batch 26 completed. Progress: 1300/4360 retailers (29.8%)
⏱️  Elapsed: 7962.7s | Est. remaining: 18742.9s

📦 Processing batch 27/88 (retailers 1301-1350)


Processing retailers:  31%|███       | 1350/4360 [2:17:38<5:03:29,  6.05s/retailer, Current=ID:64370.0, Processed=1350, Remaining=3010]

💾 Uploading batch 27 to Snowflake...
/home/ec2-user/service_account_key.json
✅ Batch 27 successfully uploaded to Snowflake!
✅ Batch 27 completed. Progress: 1350/4360 retailers (31.0%)
⏱️  Elapsed: 8267.1s | Est. remaining: 18432.5s

📦 Processing batch 28/88 (retailers 1351-1400)


Processing retailers:  32%|███▏      | 1378/4360 [2:20:30<5:01:20,  6.06s/retailer, Current=ID:64763.0, Processed=1378, Remaining=2982]

In [32]:
df_empty = pd.DataFrame({
    "main_system_id": pd.Series(dtype="Int64"),   # nullable int
    "retailer_id": pd.Series(dtype="Int64"),      # nullable int
    "address": pd.Series(dtype="string"),
    "district": pd.Series(dtype="string"),
    "state": pd.Series(dtype="string"),
    "country": pd.Series(dtype="string")
})

upload_dataframe_to_snowflake("Morocco", df_empty, "fintech", "retailer_addresses_v2", "overwrite", auto_create_table=True, conn=None)

/home/ec2-user/service_account_key.json


True

In [12]:
import pandas as pd
import requests
from time import sleep

# Initialize an empty list to store the updated data
updated_data = []
print(len(df))

# Process retailers in batches of 50
batch_size = 50


for start in range(0, len(df), batch_size):
    batch = df.iloc[start:start + batch_size]
    for index, row in batch.iterrows():
        main_system_id = row['MAIN_SYSTEM_ID']
#         cash_in_id = row['cash_in_id']
        retailer_id =  row['ID']
        lat = row['LATITUDE']
        lon = row['LONGITUDE']
        try:
            display_name, name, road, suburb, city, state, country = address_details(lat, lon)
        except Exception as e:
            print(f"Error processing retailer_id {retailer_id}: {e}")
            display_name, name, road, suburb, city, state, country = [None] * 7
        
        updated_data.append([
             main_system_id, retailer_id, display_name, city, state, country
        ])
        
#         print(updated_data)
#         sleep(1)

    df_temp = pd.DataFrame(updated_data, columns=[
       'main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
#     df_temp.to_excel('retailers_partial_output.xlsx', index=False)
    
    

# --- Step 4: Final save ---
df_updated = pd.DataFrame(updated_data, columns=['main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
upload_dataframe_to_snowflake("Morocco", df_updated, "fintech", "retailer_addresses_v2", "overwrite", auto_create_table=True, conn=None)
# df_updated.to_excel('retailers_with_regions.xlsx', index=False)
# print("✅ Exported final file: retailers_with_regions.xlsx")

5
/home/ec2-user/service_account_key.json


True

# Egypt

tablename: reatiler_addresses_v2 
schema name: fintech 
country: EG 
df: df_updated 

In [13]:
df_eg = ret_metabase("Egypt", 63420)

/home/ec2-user/service_account_key.json


In [14]:
df_eg = df_eg.head()

In [22]:
df_eg

MAIN_SYSTEM_ID       ID   LATITUDE  LONGITUDE
0          762625  1965328  30.197507  31.748868
1          878325  2162305  29.974490  31.758904
2          685304  1841414  30.239491  31.721861
3          773250  1980573  30.998901  30.669067
4          769855  1976363  26.591611  31.683381

In [15]:
# Initialize an empty list to store the updated data
updated_data_2 = []
print(len(df_eg))

# Process retailers in batches of 50
batch_size2 = 50


for start in range(0, len(df_eg), batch_size2):
    batch2 = df_eg.iloc[start:start + batch_size2]
    for index, row in batch2.iterrows():
        main_system_id2 = row['MAIN_SYSTEM_ID']
        retailer_id2 =  row['ID']
        lat2 = row['LATITUDE']
        lon2 = row['LONGITUDE']
        try:
            display_name2, name2, road2, suburb2, city2, state2, country2 = address_details(lat2, lon2)
        except Exception as e:
            print(f"Error processing retailer_id {retailer_id2}: {e}")
            display_name2, name2, road2, suburb2, city2, state2, country2 = [None] * 7
        
        updated_data_2.append([
            main_system_id2, retailer_id2, display_name2, city2, state2, country2 ])
        
        print(updated_data_2)
#         sleep(1)

    df_temp_2 = pd.DataFrame(updated_data_2, columns=[
        'main_system_id','retailer_id', 'address', 'district', 'state', 'country'])
#     df_temp.to_excel('retailers_partial_output.xlsx', index=False)
    


# --- Step 4: Final save ---
df_updated_eg = pd.DataFrame(updated_data_2, columns=[
    'main_system_id','retailer_id', 'address', 'district', 'state', 'country'
])
upload_dataframe_to_snowflake("Egypt", df_updated_eg, "fintech", "retailer_addresses_v2", "overwrite", auto_create_table=True, conn=None)
# df_updated_2.to_excel('retailers_with_regions.xlsx', index=False)
# print("✅ Exported final file: retailers_with_regions.xlsx")

5
[[508388.0, 1216203.0, 'Ras Sedr, Al Qantra Sharq Road, Qantara East, Port Said, Egypt', 'Qantara East', 'Port Said', 'Egypt']]
[[508388.0, 1216203.0, 'Ras Sedr, Al Qantra Sharq Road, Qantara East, Port Said, Egypt', 'Qantara East', 'Port Said', 'Egypt'], [918721.0, 2242866.0, 'Ad Dakahliya, 35616, Egypt', None, 'Ad Dakahliya', 'Egypt']]
[[508388.0, 1216203.0, 'Ras Sedr, Al Qantra Sharq Road, Qantara East, Port Said, Egypt', 'Qantara East', 'Port Said', 'Egypt'], [918721.0, 2242866.0, 'Ad Dakahliya, 35616, Egypt', None, 'Ad Dakahliya', 'Egypt'], [757686.0, 1959027.0, 'Soliman Alley, Giza, 12874, Egypt', None, 'Giza', 'Egypt']]
[[508388.0, 1216203.0, 'Ras Sedr, Al Qantra Sharq Road, Qantara East, Port Said, Egypt', 'Qantara East', 'Port Said', 'Egypt'], [918721.0, 2242866.0, 'Ad Dakahliya, 35616, Egypt', None, 'Ad Dakahliya', 'Egypt'], [757686.0, 1959027.0, 'Soliman Alley, Giza, 12874, Egypt', None, 'Giza', 'Egypt'], [791085.0, 2013566.0, '72, El Sheikh Abdallah El Habshy Street, Firs

True

In [7]:
# check_df = snowflake_query("Morocco", "SELECT * FROM fintech.retailer_addresses_v2;", warehouse=None, columns=[], conn=None)
check_df = check_df.sort_values(by="main_system_id", ascending=False)
check_df = check_df.drop_duplicates(subset=['main_system_id'])
check_df

main_system_id retailer_id  \
116337          81729       80971   
133615          81728       80970   
35689           81727       80969   
80184           81726       80968   
142798          81725       80967   
...               ...         ...   
142803          30164       27815   
142802          23837       51825   
142801          19092       50752   
43599           18415       51183   
142800          11837       49154   

                                                  address       district  \
116337  Had Soualem, Pachalik de Had Soualem باشوية حد...    Had Soualem   
133615  Oujeh Arous, Meknes, Pachalik de Meknes, Meknè...         Meknes   
35689   Ksar-el-Kebir, caïdat de Sidi Slama, Pachalik ...  Ksar-el-Kebir   
80184   RR303, Bir Jdid, Pachalik de Bir Jdid, El Jadi...       Bir Jdid   
142798  Lotissement El Amri, Dhar Lamhala, Oujda, Pach...          Oujda   
...                                                   ...            ...   
142803  Quartier Al Whada, Kenitra, Assaknia, Kenitra,...        Kenitra   
142802  Rue de Grenade, Harhoura, Pachalik de Harhoura...       Harhoura   
142801  Rue A2 زنقة أ 2, Hay Slimani, Hay Nahda حي الن...          Oujda   
43599   RR201, Laattaouia, Pachalik de El Attaouia, Ke...     Laattaouia   
142800  Rue 2 زنقة, Hay Karim Al Omrani, Arrondissemen...            Fez   

       state  country              dispatched_at  
116337  None  Morocco 2025-08-25 20:27:30.949961  
133615  None  Morocco 2025-08-25 20:47:57.414022  
35689   None  Morocco 2025-08-25 18:16:51.885772  
80184   None  Morocco 2025-08-25 19:36:30.032185  
142798  None  Morocco 2025-08-25 20:58:06.871341  
...      ...      ...                        ...  
142803  None  Morocco 2025-08-25 13:34:53.017788  
142802  None  Morocco 2025-08-25 13:34:53.017788  
142801  None  Morocco 2025-08-25 13:34:53.017788  
43599   None  Morocco 2025-08-25 18:32:03.844185  
142800  None  Morocco 2025-08-25 13:34:53.017788  

[7700 rows x 7 columns]

In [8]:
upload_dataframe_to_snowflake("Morocco", check_df, "fintech", "retailer_addresses_v2", "overwrite", auto_create_table=True, conn=None)

/home/ec2-user/service_account_key.json


/home/ec2-user/SageMaker/common_functions.py:503: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, _ = write_pandas(


True